In [1]:
import xarray
import numpy as np
import pandas as pd
from numba import njit

In [3]:
# ### States
path_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/states.nc'
luh2_states = xarray.open_dataset(path_LUH2, engine="netcdf4", decode_times=False)
luh2_states_worldwide = luh2_states.isel(time=slice(1000, 1173))
luh2_states_worldwide['time'] = pd.date_range(start="1850-01-01", end="2022-01-01", freq='YS')

In [4]:
luh2_added_states_worldwide = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-IMAGE-ssp119-2-1-f_gn_2015-2100.nc",\
                                        engine="netcdf4", decode_times=False)
luh2_added_trans = luh2_added_states_worldwide.drop_vars(['lat_bounds', 'lon_bounds', 'time_bnds'])
luh2_added_trans.coords['time'] = pd.date_range(start='2015-01-01', end='2100-01-01', freq='YS')

# ### Static

staticcode = 'D:/kerja/asisten riset/code v/gpw_v4_national_identifier_grid_rev11_15_min_finall.nc'
luh2_static = xarray.open_dataset(staticcode, engine="netcdf4")

country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code.xlsx", engine="openpyxl")
ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['name'])

ccode_convert = np.zeros((720, 1440), dtype="<U64")

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh2_static['ccode'].to_numpy().astype('int64')

for i in range(720):
    for j in range(1440):
        if ccode_worldwide_int[i][j] in ccode_dict.keys():
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "ocean"

country = xarray.Dataset({"country": (["lat", "lon"], ccode_convert)},
                          coords={ "lon":luh2_states_worldwide.coords["lon"].to_numpy(), "lat":luh2_states_worldwide.coords["lat"].to_numpy()})

In [5]:
crop_class = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Crop Classification_latest.xlsx", engine="openpyxl", skiprows=1)
crop_class = crop_class.drop('Unnamed: 0', axis=1)
crop_class.rename(columns={'FAO Crops': 'Item'}, inplace=True)

fao_stat = pd.read_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Production_Crops_Livestock_E_All_Data_analyze.csv")
fao_stat_area_harv = fao_stat[fao_stat['Element'] == "Area harvested"]
fao_stat_area_harv.insert(9, 'True Value', fao_stat_area_harv['Value']/100, True)
faostat_area_harvested = fao_stat_area_harv.merge(crop_class, on='Item')

In [3]:
gfrac = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/GFRAC_GRAPC_Interpolation/GFRAC-RESAMPLE-NEW-1970-2100.nc", engine='netcdf4')

gfrac_ngfbfc = gfrac.coords['NGFBFC'].data.tolist()
gfrac_ngfbfc_ir_rf = ["grass"]
for gfrac_cls in gfrac_ngfbfc:
    if  gfrac_cls[0:2] == 'RF' or gfrac_cls[0:2] == 'IR':
        gfrac_ngfbfc_ir_rf.append(gfrac_cls[3:])

gfrac_ngfbfc_ir_rf_list = gfrac_ngfbfc_ir_rf[:17]

In [6]:
# gfracn_new = np.zeros((17, 27, 720, 1440), dtype="float32")

# for ngfbfc_frac_idx, ngfbfc_frac in enumerate(gfrac_ngfbfc_ir_rf_list):
#     try:
#         if ngfbfc_frac == "grass":
#             gfracn_new[ngfbfc_frac_idx] = gfrac["GFRAC_res"].sel(NGFBFC=ngfbfc_frac)
#         else:
#             gfracn_new[ngfbfc_frac_idx] = gfrac["GFRAC_res"].sel(NGFBFC=f"RF {ngfbfc_frac}") + gfrac["GFRAC_res"].sel(NGFBFC=f"IR {ngfbfc_frac}")
            
#     except KeyError or ValueError:
#         gfracn_new[ngfbfc_frac_idx] = gfrac["GFRAC_res"].sel(NGFBFC=f"RF {ngfbfc_frac}") + gfrac["GFRAC_res"].sel(NGFBFC='IR Tropical oilcrops')

In [6]:
# gfrac_ngfbfc = [element.strip() for element in gfrac_ngfbfc_ir_rf_list]

In [8]:
# #checkpoint 1

# gfrac_new_netcdf = xarray.Dataset({
#     "GFRAC_resnew":(["NGFBFC", "time", "lat", "lon"], gfracn_new)
# },coords={
#         "time": pd.date_range(start='1970-01-01', end='2100-01-01', freq='5YS'),
#         "lat": gfrac.coords["lat"].to_numpy(),
#         "lon": gfrac.coords["lon"].to_numpy(),
#         "NGFBFC": list(gfrac_ngfbfc)
#     })
# gfrac_new_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunarc/cadlan/GFRAC_GRAPC_Interpolation/GFRAC-RESAMPLE-NEW-1970-2100_v2.nc", mode="w", format="NETCDF4")

In [6]:
gfracnew = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/GFRAC_GRAPC_Interpolation/GFRAC-RESAMPLE-NEW-1970-2100_v2.nc", engine="netcdf4")
gfracnew_ngfbfc = gfracnew.coords['NGFBFC'].data.tolist()

In [7]:
# fungsi interpolasi
@njit
def grid_level(kolom_sebelumnya, percent):
    return np.divide(np.multiply(kolom_sebelumnya, percent), 100)

@njit
def percent_change(gfrac, kolom_sekarang, kolom_sebelumnya):
    result = np.multiply(np.divide(kolom_sekarang, kolom_sebelumnya), 100)
    return grid_level(gfrac, result)

In [8]:
arr = np.zeros((6, 17, 720, 1440), dtype="float32")

for gfrac_idx, gfrac_class in enumerate(gfracnew_ngfbfc):
    arr[0][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=0, NGFBFC=gfrac_idx)
    arr[5][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=1, NGFBFC=gfrac_idx)
    # arr[10][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=2, NGFBFC=gfrac_idx)
    # arr[15][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=3, NGFBFC=gfrac_idx)
    # arr[20][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=4, NGFBFC=gfrac_idx)
    # arr[25][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=5, NGFBFC=gfrac_idx)
    # arr[30][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=6, NGFBFC=gfrac_idx)
    # arr[35][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=7, NGFBFC=gfrac_idx)
    # arr[40][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=8, NGFBFC=gfrac_idx)
    # arr[45][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=9, NGFBFC=gfrac_idx)
    # arr[50][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=10, NGFBFC=gfrac_idx)
    # arr[55][gfrac_idx] = gfracnew['GFRAC_resnew'].isel(time=11, NGFBFC=gfrac_idx)

In [9]:
years = np.linspace(1970, 1975, 6, dtype=np.int64)
unique_countries = np.unique(ccode_convert)

for country in unique_countries:
    for column_idx, column in enumerate(years[1:]):
        # Filter data FAO satu kali untuk setiap tahun
        current_year_fao = faostat_area_harvested[(faostat_area_harvested['Year'] == column) & 
                                                   (faostat_area_harvested['Area'] == country)]
        previous_year_fao = faostat_area_harvested[(faostat_area_harvested['Year'] == column - 1) & 
                                                    (faostat_area_harvested['Area'] == country)]
        
        for gfrac_idx, gfrac_class in enumerate(gfracnew_ngfbfc):
            current_year_fao_sum = current_year_fao[current_year_fao['IMAGE Classification'] == gfrac_class]['True Value'].sum()
            previous_year_fao_sum = previous_year_fao[previous_year_fao['IMAGE Classification'] == gfrac_class]['True Value'].sum()
            
            current_year_gfrac = arr[column_idx + 1][gfrac_idx]
            previous_year_gfrac = arr[column_idx][gfrac_idx]
            
            if np.isnan(current_year_gfrac).any() or current_year_gfrac.any() == 0.0:
                arr[column_idx + 1][gfrac_idx] = percent_change(previous_year_gfrac, current_year_fao_sum, previous_year_fao_sum)


In [15]:
#checkpoint 2
gfrac_interp_netcdf = xarray.Dataset({
    "GFRAC_interp": (["time", "NGFBFC", "lat", "lon"], arr) 
},
    coords={
        "time": pd.date_range("1970-01-01", "1975-01-01", freq='YS'),
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy(),
        "NGFBFC": gfracnew_ngfbfc,
})

# coords = ("time", "lat", "lon", "NGFBFC")
# data_vars = {
#     ngfbc: (coords, arr_intrp[i]) for i, ngfbc in enumerate(gfracnew_ngfbfc)
# }
# gfrac_interp_netcdf = gfrac_interp_netcdf.assign(data_vars)
gfrac_interp_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunarc/cadlan/GFRAC_GRAPC_Interpolation/GFRAC-INTERPOLATION-WORLDWIDE-1970_1975.nc", mode='w', format="NETCDF4")

In [16]:
df = gfrac_interp_netcdf.to_dataframe()

In [17]:
table = pd.pivot_table(df, values=gfracnew_ngfbfc, index=['country'], columns=['time'], aggfunc="sum", fill_value=0)

In [18]:
df_index = table.stack(level=0)
data = pd.to_datetime(df_index.columns, format='%d/%m/%Y %H.%M.%S')
df_index.columns = data.year
df_index.index.set_names(['country', 'type'], inplace=True)
df_index = df_index.reset_index()
df_index

time,country,type,1970,1971,1972,1973,1974,1975
0,Afghanistan,Maize,59.298950,42.290565,2.674874,4.313183,2.804712,6.857570
1,Afghanistan,Oil & palm fruit,0.000000,0.000000,1.285459,0.000000,2.040646,0.000000
2,Afghanistan,Other non-food & luxury & spices,4.883850,5.042311,0.257842,14.468429,0.332324,0.755558
3,Afghanistan,Other temperate cereals,0.000000,1.980394,0.000000,0.000000,0.000000,73.104248
4,Afghanistan,Plant based fibres,0.070224,1.286835,0.000000,6.695700,2.269390,16.320709
...,...,...,...,...,...,...,...,...
3956,Åland Islands,Tropical oil crops,0.000164,0.000000,0.013879,0.003460,0.000000,0.002617
3957,Åland Islands,Tropical roots & tubers,0.000000,0.000000,0.000000,0.011530,0.000026,0.000166
3958,Åland Islands,Vegetables & fruits,0.000000,0.071658,0.010937,0.000000,0.000175,0.000000
3959,Åland Islands,Wheat,0.002691,0.000000,0.002414,0.000000,0.002219,0.000000


In [ ]:
df_index.to_excel("",index=False)